# Using C#

This notebook shows the use of Collective2 data in a Windows application.

![Image](./WindowsDesktopApp_AllData.png)

![Image](./WindowsDesktopApp_ScoreData.png)

And this is C# code:

~~~csharp
using C2StatsPg;
using System;
using System.Collections.Generic;
using System.Data;
using System.Linq;
using System.Windows.Forms;

namespace ScoringWorkbenchDemo
{
    public partial class Form1 : Form
    {
        public Form1()
        {
            InitializeComponent();
        }

        private void button1_Click(object sender, EventArgs e)
        {
            showData();
        }

        private IEnumerable<C2scoreScoringWorkbenchDataCached> getScoringWorkbenchDataAll()
        {
            // Connect to the Collective2 public database and return ScoringWorkbench data.
            string server_url = "quantdata.collective2.com";
            using (C2StatsPgDb db = new C2StatsPgDb($"server={server_url};user id=c2public;password=c2public;database=postgres"))
            {
                return (from s in db.ScoringWorkbenchView orderby s.Strategyid select s).ToList();
            }
        }

        /// <summary>
        /// Show data in grids.
        /// </summary>
        private void showData()
        {
            // Get all data
            IEnumerable<C2scoreScoringWorkbenchDataCached> data = getScoringWorkbenchDataAll();
            // Show the data in the first grid
            allDataGridView.DataSource = data;
            // Calculate a score
            IEnumerable<ScoreStructured> score = calculateScore(data);
            // Show the score in the second grid
            scoreDataGridView.DataSource = score.ToList();
        }

        /// <summary>
        /// A demo function for the risk calculation.
        /// </summary>
        /// <param name="strat">Strategy data</param>
        /// <returns>Risk score</returns>
        private double? riskFun(C2scoreScoringWorkbenchDataCached strat)
        {
            return
              (20 - strat.Maxdrawdownpcnt * (20 / 45))
            + (strat.Maxworstlosspercentequity060 == null ? 0 : (20 - strat.Maxworstlosspercentequity060 * (20.0 / 8)))
            + (10 - strat.Optionpercent * (10 / 1))
            + (strat.Shortoptionscovered >= 100 ? 10 : strat.Shortoptionscovered / 100 * 10);
        }

        /// <summary>
        /// A demo function for the behavior calculation.
        /// </summary>
        /// <param name="strat">Strategy data</param>
        /// <returns>Behavior score</returns>
        private double? behavioralFun(C2scoreScoringWorkbenchDataCached strat)
        {
            return
              (strat.Agedays >= 720 ? 40 : strat.Agedays / 720 * 40)
            + (20 - strat.Dailymaxlevmax * (20 / 10))
            + (strat.Tradedaysago <= 60 ? 20 : 0)
            + (strat.Pcntmonthsprofitable >= 100 ? 5 : strat.Pcntmonthsprofitable / 100 * 5)
            + (strat.Numtrades >= 100 ? 20 : strat.Numtrades / 100 * 20)
            + ((strat.Deltaequity90days <= 0 && strat.Deltaequity90days > -7) ? 5 : 0); 
        }

        /// <summary>
        /// A demo function for the profit calculation.
        /// </summary>
        /// <param name="strat">Strategy data</param>
        /// <returns>Profit score</returns>
        private double? profitabilityFun(C2scoreScoringWorkbenchDataCached strat)
        {
            return strat.Annreturn > 0.6 ? 20 : strat.Annreturn / 0.6 * 20;
        }

        /// <summary>
        /// Calculate score for all strategies in the collection.
        /// </summary>
        /// <param name="data"></param>
        /// <returns></returns>
        private IEnumerable<ScoreStructured> calculateScore(IEnumerable<C2scoreScoringWorkbenchDataCached> data)
        {
            return from strat in data
                   // Selection
                   where !(strat.Agedays < 90 || strat.Annreturn < 0 || strat.Deltaequity180days < 0)
                   // Calculations
                   let risk = (decimal)Math.Round(riskFun(strat).Value, 2)
                   let behav = (decimal)Math.Round(behavioralFun(strat).Value, 2)
                   let profit = (decimal)Math.Round(profitabilityFun(strat).Value, 2)
                   let score = (decimal)(risk + behav + profit)
                   // Order by the score down.
                   orderby score descending
                   // Format a result
                   select new ScoreStructured()
                   {
                       Added = strat.Added,
                       Score = score,
                       ScoreRisk = risk,
                       ScoreBehavior = behav,
                       ScoreProfitability = profit,
                       StrategyId = strat.Strategyid,
                       StrategyName = strat.Strategyname,
                   };
        }

        /// <summary>
        /// A structure used for UI.
        /// </summary>
        public class ScoreStructured
        {
            public decimal Score { get; set; }
            public long StrategyId { get; set; }
            public string StrategyName { get; set; }
            public DateTime Added { get; set; }
            public decimal ScoreRisk { get; set; }
            public decimal ScoreBehavior { get; set; }
            public decimal ScoreProfitability { get; set; }
        }

    }
}
~~~